In [10]:
from collections import defaultdict
from datetime import datetime

# Stores data minute-wise per expiry
mega_data_by_expiry = defaultdict(list)

# Stores current buffer per expiry
minute_buffer_by_expiry = defaultdict(lambda: {
    "nifty": {},
    "options": defaultdict(lambda: {"CE": {}, "PE": {}})
})

current_minute = None
last_predicted_time_by_expiry = {}


In [11]:
# === GLOBAL EXPIRY CONFIG ===
EXPIRY_LIST = [ "17JUL25","31JUL25",  "24JUL25"]


In [12]:
import re

def detect_expiry_from_symbol(symbol):
    match = re.match(r"NIFTY(\d{2}[A-Z]{3}\d{2})\d{5}(CE|PE)", symbol)
    if match:
        return match.group(1)  # e.g., "04JUL25"
    return None

def process_tick(market_data):
    global current_minute, mega_data_by_expiry, minute_buffer_by_expiry, nifty_spot

    now = datetime.now().replace(second=0, microsecond=0)

    if current_minute != now:
        for expiry, buffer in minute_buffer_by_expiry.items():
            if buffer["nifty"]:  # Ensure it has data
                mega_data_by_expiry[expiry].append({
                    "time": current_minute,
                    "nifty": buffer["nifty"],
                    "options": dict(buffer["options"])
                })
                # Reset buffer for next minute
                minute_buffer_by_expiry[expiry] = {
                    "nifty": {},
                    "options": defaultdict(lambda: {"CE": {}, "PE": {}})
                }
        current_minute = now

    for symbol, info in market_data.items():
        if symbol == "NIFTY":
            for expiry in minute_buffer_by_expiry:
                update_ohlc(minute_buffer_by_expiry[expiry]["nifty"], info)
            if "price" in info:
                nifty_spot = info["price"]

        elif symbol.startswith("NIFTY"):
            expiry = detect_expiry_from_symbol(symbol)
            if not expiry:
                continue
            strike = symbol[12:-2]
            option_type = symbol[-2:]
            update_ohlc(minute_buffer_by_expiry[expiry]["options"][strike][option_type], info)

def update_ohlc(ohlc, info):
    price = info.get("price")
    if price is not None:
        if "open" not in ohlc:
            ohlc["open"] = ohlc["high"] = ohlc["low"] = ohlc["close"] = price
        else:
            ohlc["high"] = max(ohlc["high"], price)
            ohlc["low"] = min(ohlc["low"], price)
            ohlc["close"] = price

    for key in ["bbPrice", "bsPrice", "dayVolume", "openInterest"]:
        if key in info:
            ohlc[key] = info[key]
            
def predict_from_latest_minute():
    for expiry, data in mega_data_by_expiry.items():
        if not data:
            continue

        latest_minute = data[-1]
        if last_predicted_time_by_expiry.get(expiry) == latest_minute["time"]:
            continue  # already predicted

        if "nifty" not in latest_minute or "price" not in latest_minute["nifty"]:
            continue

        current_price = latest_minute["nifty"]["price"]
        nearest_strike = round(current_price / 50) * 50
        strike_range = [str(nearest_strike + i * 50) for i in range(-3, 4)]

        expiry_dt = datetime.strptime(expiry, "%d%b%y").replace(hour=15, minute=30)

        print(f"\n🧠 Predicting for {expiry} at {latest_minute['time'].strftime('%H:%M:%S')}")
        for strike in strike_range:
            for opt_type in ["CE", "PE"]:
                if strike in latest_minute["options"]:
                    try:
                        label = predict_xgb_action(strike, opt_type, expiry_dt, latest_minute)
                        if label in ["BUY", "STRONG BUY"]:
                            print(f"✅ {strike}{opt_type}: {label}")
                    except Exception as e:
                        print(f"❌ Error predicting {strike}{opt_type}:", e)

        last_predicted_time_by_expiry[expiry] = latest_minute["time"]


In [13]:
# ============================
# Block 1: WebSocket + Live Option Table (Multi-Expiry)
# ============================

import socketio
import nest_asyncio
import asyncio
import threading
import time
from datetime import datetime
from IPython.display import display, DisplayHandle

# Jupyter fix
nest_asyncio.apply()

# === Global Variables ===
sio = socketio.AsyncClient()
latest_data = {}
nifty_spot = None
all_expiries = EXPIRY_LIST # Add more if needed
last_data_time = time.time()
display_handle = DisplayHandle()
display_handle.display("")  # Create empty output cell
selected_expiry = EXPIRY_LIST[0]  # default display expiry only
range_count = 5


# ============================
# Display Live Table
# ============================
def display_table():
    output = []
    output.append(f"📈 Live NIFTY Option Prices (Last updated: {datetime.now().strftime('%H:%M:%S')})\n")
    output.append("{:<10} {:>8} {:>8} {:>8} {:>8}   {:>8} {:>8} {:>8} {:>8}".format(
        "Strike", "CE", "Bid", "Ask", "Vol", "PE", "Bid", "Ask", "Vol"))
    output.append("-" * 80)

    if nifty_spot is None:
        output.append("⏳ Waiting for NIFTY price...")
        display_handle.update(output)
        return

    nearest_strike = round(nifty_spot / 50) * 50
    strikes = [nearest_strike + i * 50 for i in range(-5, 6)]

    for strike in strikes:
        ce_key = f"NIFTY{selected_expiry}{strike}CE"
        pe_key = f"NIFTY{selected_expiry}{strike}PE"
        ce = latest_data.get(ce_key, {})
        pe = latest_data.get(pe_key, {})

        def fmt(d, key): return str(d.get(key, "--")).rjust(8)

        row = "{:<10} {} {} {} {}   {} {} {} {}".format(
            strike,
            fmt(ce, "price"), fmt(ce, "bbPrice"), fmt(ce, "bsPrice"), fmt(ce, "dayVolume"),
            fmt(pe, "price"), fmt(pe, "bbPrice"), fmt(pe, "bsPrice"), fmt(pe, "dayVolume")
        )
        output.append(row)

    display_handle.update(output)


# ============================
# WebSocket Events
# ============================
@sio.event
async def connect():
    print("✅ Connected to InstaOptions")
    await sio.emit("subscribe", [[None, "NIFTY"], 100])  # Subscribe NIFTY spot
    await auto_subscribe_all_expiries(all_expiries, range_count)


@sio.on("ticker:tick")
async def on_tick(data):
    global nifty_spot
    update_data_time()
    for symbol, info in data.get("marketData", {}).items():
        latest_data[symbol] = info
        if symbol == "NIFTY" and "price" in info:
            nifty_spot = info["price"]
    process_tick(data.get("marketData", {}))  # Your custom logic here


@sio.event
async def disconnect():
    print("🔌 Disconnected from server")


@sio.event
async def connect_error(data):
    print("❌ Connection error:", data)


# ============================
# Subscription Logic
# ============================
async def auto_subscribe_all_expiries(expiries, range_count=5):
    while "NIFTY" not in latest_data or "price" not in latest_data["NIFTY"]:
        print("⏳ Waiting for NIFTY spot price...")
        await asyncio.sleep(1)

    nifty_price = latest_data["NIFTY"]["price"]
    nearest = round(nifty_price / 50) * 50
    strikes = [nearest + i * 50 for i in range(-range_count, range_count + 1)]

    for expiry in expiries:
        symbols = [f"NIFTY{expiry}{strike}{opt}" for strike in strikes for opt in ["CE", "PE"]]
        print(f"📡 Subscribing {len(symbols)} symbols for expiry {expiry}")
        await sio.emit("subscribe", [[None] + symbols, 100])


# ============================
# Refresh UI Periodically
# ============================
def start_socket_loop():
    async def main():
        async def refresh():
            while True:
                display_table()
                await asyncio.sleep(3)

        refresh_task = asyncio.create_task(refresh())

        await sio.connect(
            'https://www.instaoptions.in',
            transports=['websocket'],
            socketio_path='analytics-ticker/socket.io'
        )
        await sio.wait()
        refresh_task.cancel()

    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(main())


# ============================
# Watchdog: Auto Reconnect
# ============================
def update_data_time():
    global last_data_time
    last_data_time = time.time()

def start_watchdog():
    async def watchdog_loop():
        while True:
            await asyncio.sleep(10)
            elapsed = time.time() - last_data_time
            if elapsed > 15:
                print("⚠️ No tick updates in 15s. Retrying subscriptions...")
                try:
                    await auto_subscribe_all_expiries(all_expiries, range_count)
                except Exception as e:
                    print("❌ Retry failed:", e)
                update_data_time()

    loop = asyncio.new_event_loop()
    threading.Thread(target=lambda: loop.run_until_complete(watchdog_loop()), daemon=True).start()


# ============================
# Start Everything
# ============================
start_watchdog()
threading.Thread(target=start_socket_loop, daemon=True).start()


['📈 Live NIFTY Option Prices (Last updated: 18:30:27)\n',
 'Strike           CE      Bid      Ask      Vol         PE      Bid      Ask      Vol',
 '--------------------------------------------------------------------------------',
 '24850            --       --       --       --         --       --       --       --',
 '24900            --       --       --       --         --       --       --       --',
 '24950            --       --       --       --         --       --       --       --',
 '25000            --       --       --       --         --       --       --       --',
 '25050            --       --       --       --         --       --       --       --',
 '25100            --       --       --       --         --       --       --       --',
 '25150            --       --       --       --         --       --       --       --',
 '25200            --       --       --       --         --       --       --       --',
 '25250            --       --       --       --        

✅ Connected to InstaOptions
⏳ Waiting for NIFTY spot price...
📡 Subscribing to 23 symbols for expiry 17JUL25
📡 Subscribing to 23 symbols for expiry 31JUL25
📡 Subscribing to 23 symbols for expiry 24JUL25
🔌 Disconnected from server
✅ Connected to InstaOptions
📡 Subscribing to 23 symbols for expiry 17JUL25
📡 Subscribing to 23 symbols for expiry 31JUL25
📡 Subscribing to 23 symbols for expiry 24JUL25


packet queue is empty, aborting


🔌 Disconnected from server
✅ Connected to InstaOptions
📡 Subscribing to 23 symbols for expiry 17JUL25
📡 Subscribing to 23 symbols for expiry 31JUL25
📡 Subscribing to 23 symbols for expiry 24JUL25
⚠️ No tick updates in 15s. Retrying subscriptions...
📡 Subscribing to 23 symbols for expiry 17JUL25
📡 Subscribing to 23 symbols for expiry 31JUL25
📡 Subscribing to 23 symbols for expiry 24JUL25
⚠️ No tick updates in 15s. Retrying subscriptions...
📡 Subscribing to 23 symbols for expiry 17JUL25
📡 Subscribing to 23 symbols for expiry 31JUL25
📡 Subscribing to 23 symbols for expiry 24JUL25
⚠️ No tick updates in 15s. Retrying subscriptions...
📡 Subscribing to 23 symbols for expiry 17JUL25
📡 Subscribing to 23 symbols for expiry 31JUL25
📡 Subscribing to 23 symbols for expiry 24JUL25
⚠️ No tick updates in 15s. Retrying subscriptions...
📡 Subscribing to 23 symbols for expiry 17JUL25
📡 Subscribing to 23 symbols for expiry 31JUL25
📡 Subscribing to 23 symbols for expiry 24JUL25
⚠️ No tick updates in 15s. R

packet queue is empty, aborting



📡 Subscribing to 23 symbols for expiry 17JUL25
📡 Subscribing to 23 symbols for expiry 31JUL25
📡 Subscribing to 23 symbols for expiry 24JUL25
🔌 Disconnected from server
✅ Connected to InstaOptions
📡 Subscribing to 23 symbols for expiry 17JUL25
📡 Subscribing to 23 symbols for expiry 31JUL25
📡 Subscribing to 23 symbols for expiry 24JUL25
⚠️ No tick updates in 15s. Retrying subscriptions...
📡 Subscribing to 23 symbols for expiry 17JUL25
📡 Subscribing to 23 symbols for expiry 31JUL25
📡 Subscribing to 23 symbols for expiry 24JUL25
🔌 Disconnected from server
✅ Connected to InstaOptions
📡 Subscribing to 23 symbols for expiry 17JUL25
📡 Subscribing to 23 symbols for expiry 31JUL25
📡 Subscribing to 23 symbols for expiry 24JUL25
⚠️ No tick updates in 15s. Retrying subscriptions...
📡 Subscribing to 23 symbols for expiry 17JUL25
📡 Subscribing to 23 symbols for expiry 31JUL25
📡 Subscribing to 23 symbols for expiry 24JUL25
⚠️ No tick updates in 15s. Retrying subscriptions...
📡 Subscribing to 23 symbo

In [14]:
import pandas as pd
import os
import pandas as pd
import os

def save_mega_data_to_csv(data, filename):
    flat = []
    for entry in data:
        t = entry['time']
        flat.append({'time': t, 'symbol': 'NIFTY', **entry['nifty']})
        for strike, opts in entry['options'].items():
            for opt_type, values in opts.items():
                flat.append({'time': t, 'symbol': f"{strike}{opt_type}", **values})

    df = pd.DataFrame(flat)

    if os.path.exists(filename):
        existing = pd.read_csv(filename, parse_dates=["time"])
        df = df[~df['time'].isin(existing['time'])]
        df = pd.concat([existing, df], ignore_index=True).drop_duplicates(subset=["time", "symbol"])

    df.to_csv(filename, index=False)
    print(f"✅ Data saved to {filename}")

def save_all_expiry_csvs():
    for expiry, data in mega_data_by_expiry.items():
        filename = f"Data/nifty_option_megadata_{expiry}_{datetime.today().strftime('%Y-%m-%d')}.csv"
        save_mega_data_to_csv(data, filename)

import threading
import time

def periodic_saver(interval_minutes=1):
    while True:
        try:
            save_all_expiry_csvs()
        except Exception as e:
            print("❌ Auto-save failed:", e)
        time.sleep(interval_minutes * 60)

threading.Thread(target=periodic_saver, daemon=True).start()

✅ Data saved to Data/nifty_option_megadata_17JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_31JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_24JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_17JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_31JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_24JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_17JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_31JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_24JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_17JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_31JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_24JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_17JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_31JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_24JUL25_2025-07-14.csv
✅ Data sav

In [15]:
async def auto_subscribe_all_expiries(expiries=EXPIRY_LIST, range_count=3):
    while "NIFTY" not in latest_data or "price" not in latest_data["NIFTY"]:
        print("⏳ Waiting for NIFTY spot price...")
        await asyncio.sleep(1)

    nifty_spot = latest_data["NIFTY"]["price"]
    nearest = round(nifty_spot / 50) * 50
    strike_list = [nearest + i * 50 for i in range(-range_count, range_count + 1)]

    for expiry in expiries:
        symbols = [f"NIFTY{expiry}{strike}{t}" for strike in strike_list for t in ["CE", "PE"]]
        symbols.insert(0, "NIFTY")  # include NIFTY spot

        print(f"📡 Subscribing to {len(symbols)} symbols for expiry {expiry}")
        await sio.emit("subscribe", [[None] + symbols, 100])


In [16]:
await auto_subscribe_all_expiries()

⏳ Waiting for NIFTY spot price...
⏳ Waiting for NIFTY spot price...
📡 Subscribing to 15 symbols for expiry 17JUL25
📡 Subscribing to 15 symbols for expiry 31JUL25
📡 Subscribing to 15 symbols for expiry 24JUL25


In [17]:
import threading
import time

def start_background_tasks():
    def prediction_loop():
        while True:
            predict_from_latest_minute()
            time.sleep(5)  # Predict every 5s or 60s if you prefer

    def saver_loop():
        while True:
            save_all_expiry_csvs()
            time.sleep(60)

    threading.Thread(target=prediction_loop, daemon=True).start()
    threading.Thread(target=saver_loop, daemon=True).start()


In [ ]:
start_background_tasks()

✅ Data saved to Data/nifty_option_megadata_17JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_31JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_24JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_17JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_31JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_24JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_17JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_31JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_24JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_17JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_31JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_24JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_17JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_31JUL25_2025-07-14.csv
✅ Data saved to Data/nifty_option_megadata_24JUL25_2025-07-14.csv
✅ Data sav